# Prelude

In [1]:
import numpy
import scipy
import matplotlib
import seaborn

import torch
import torch.nn as nn
import datasets
import wandb


import nltk.lm as lm
from nltk import ngrams
from nltk import tokenize
from nltk.corpus import stopwords, gutenberg
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize

from matplotlib import pyplot

In [2]:
import pickle

In [3]:
dataset = datasets.load_dataset(
    'cnn_dailymail',
    '3.0.0',
)

# Let's print something the shape or whatever

In [5]:
train = dataset['train']
test = dataset['test']
val = dataset['validation']

In [7]:
print(train.shape)
print(test.shape)
print(val.shape)

(287113, 3)
(11490, 3)
(13368, 3)


In [44]:
print(str(train.data[0][0])[:100])
print(str(train.data[1][0])[:100])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 milli
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has


# Fun with NLTK

In [3]:
# train_text = [tokenize.word_tokenize(str(x)) for x in train.data[0]]
# train_high = [tokenize.word_tokenize(str(x)) for x in train.data[1]]

In [4]:
# pickle.dump(train_text, open('train_text.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)
# pickle.dump(train_high, open('train_high.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)

In [5]:
train_text_raw = pickle.load(open('train_text.pickle', 'rb'))
train_high_raw = pickle.load(open('train_high.pickle', 'rb'))

In [7]:
stop_words = set(stopwords.words('english'))
punctuation = set(x for x in train_text_raw[0] + train_high_raw[0] if not any(y.isalpha() or y.isdigit() for y in x))

In [8]:
def clean(text):
    return [
        word.lower()
        for word in text
        if word.lower() not in stop_words and word not in punctuation
    ]

train_text = [clean(x) for x in train_text_raw]
train_high = [clean(x) for x in train_high_raw]